In [60]:
# <a name="paso-0"></a> Paso 0: Configurar rutas del sistema

"""
ALGORITMO GENÉTICO PARA ENRUTAMIENTO DE VEHÍCULOS (VRPTW)

Este es el PRIMER PASO. Ejecuta esta celda antes que cualquier otra.
Configura las rutas del sistema para que Python encuentre los módulos necesarios.
"""

import sys
import os

# Asegurar que estamos en el directorio correcto
os.chdir(r'C:\Users\sjaim\Metahuristica')
sys.path.insert(0, r'C:\Users\sjaim\Metahuristica')

print("="*80)
print(" " * 20 + "ALGORITMO GENÉTICO PARA ENRUTAMIENTO DE VEHÍCULOS")
print("="*80)
print("\nPASO 0: SISTEMA CONFIGURADO")
print("-" * 80)
print(f"\nWorking directory: {os.getcwd()}")
print(f"System path: {sys.path[0]}")
print(f"\nSiguiente paso: Ejecuta la siguiente celda (Paso 1)")
print("-" * 80)

                    ALGORITMO GENÉTICO PARA ENRUTAMIENTO DE VEHÍCULOS

PASO 0: SISTEMA CONFIGURADO
--------------------------------------------------------------------------------

Working directory: C:\Users\sjaim\Metahuristica
System path: C:\Users\sjaim\Metahuristica

Siguiente paso: Ejecuta la siguiente celda (Paso 1)
--------------------------------------------------------------------------------


# Algoritmo Genético para Enrutamiento de Vehículos con Ventanas de Tiempo (VRPTW)

## Tabla de Contenidos

| Paso | Sección | Descripción |
| :---: | :--- | :--- |
| 0 | **[Sistema](#paso-0)** | Configuración del entorno de trabajo |
| 1 | **[Librerías](#paso-1)** | Importación de módulos necesarios |
| 2 | **[Datos](#paso-2)** | Carga de la instancia del problema |
| 3 | **[Análisis](#paso-3)** | Visualización de datos de camiones y clientes |
| 4 | **[Configuración GA](#paso-4)** | Parámetros del Algoritmo Genético |
| 5 | **[Operadores](#paso-5)** | Funciones auxiliares y operadores genéticos |
| 6 | **[Ejecución](#paso-6)** | Inicialización y ejecución del GA |
| 7 | **[Intensificación](#paso-7)** | Optimización adicional de la mejor solución |
| 8 | **[Resultados](#paso-8)** | Análisis detallado de la solución final |

---

## ¿Qué es el VRPTW?

El **Problema de Enrutamiento de Vehículos con Ventanas de Tiempo (VRPTW)** consiste en diseñar rutas óptimas para una flota de vehículos que deben visitar un conjunto de clientes, respetando:

- **Ventanas de tiempo**: Cada cliente debe ser visitado dentro de un horario específico
- **Capacidad de vehículos**: Los camiones tienen límites de carga
- **Restricciones operativas**: Horarios de almuerzo, tiempo máximo de jornada, disponibilidad de muelles

### Objetivo

**Minimizar el costo total** que incluye:
- Costos de contratación de vehículos (por hora o franjas fijas)
- Penalizaciones por llegadas tempranas o tardías
- Penalizaciones por violaciones operativas

### Metodología

Se utiliza un **Algoritmo Genético** que:
1. Crea una población de soluciones candidatas
2. Evoluciona estas soluciones mediante selección, cruce y mutación
3. Aplica búsqueda local para refinar soluciones
4. Mantiene diversidad para explorar el espacio de soluciones

5. Intensifica la búsqueda en las mejores soluciones encontradas```

dat_file = r'ruta\a\tu\archivo.dat'

---```python

En el Paso 2, cambia la ruta:

## Instrucciones de Uso

### Modificar Archivo de Datos

### Primera Ejecución

- **Total estimado**: 3-7 minutos

1. **Ejecuta el Paso 0**: Configura las rutas del sistema- **Intensificación**: 1-2 minutos

2. **Ejecuta el Paso 1**: Carga las librerías necesarias- **Ejecución del GA**: 1-3 minutos (depende del tamaño de la instancia)

3. **Modifica la ruta del archivo** en el Paso 2 si deseas usar otra instancia- **Inicialización del GA**: 1-2 segundos

4. **Ejecuta todas las celdas en orden** desde el Paso 2 hasta el Paso 8- **Carga de datos**: < 1 segundo


### Tiempo de Ejecución

---

## Estructura del Notebook

### Fase 1: Preparación (Pasos 0-3)
- Configuración del entorno
- Carga y análisis de datos
- Visualización de la instancia del problema

### Fase 2: Configuración del GA (Pasos 4-5)
- Definición de parámetros del algoritmo
- Implementación de operadores genéticos
- Funciones auxiliares y de evaluación

### Fase 3: Optimización (Paso 6)

- Creación de población inicial con heurísticas inteligentes- Resumen ejecutivo de la solución

- Evolución de soluciones mediante el Algoritmo Genético- Análisis de costos y penalizaciones

- Aplicación de búsqueda local y control de diversidad- Visualización detallada de rutas

### Fase 5: Resultados (Paso 8)

### Fase 4: Refinamiento (Paso 7)

- Intensificación de la mejor solución encontrada- Path relinking entre soluciones de élite
- Técnicas de mejora local (2-opt, reubicación, intercambio)

In [61]:
# <a name="paso-1"></a> Paso 1: Importar librerías necesarias

"""
Importamos todos los módulos necesarios para:
1. Cargar y procesar datos (data_loader)
2. Codificar/decodificar soluciones (encoding)
3. Evaluar soluciones (simulator)
4. Aplicar búsqueda local (ga_utils)
5. Visualizar resultados (matplotlib, pandas)
"""

from src.data_loader import parse_ampl_dat, build_instance
from src.encoding import (
    encode_routes, decode_vector, route_based_crossover, 
    swap_mutation, insert_mutation, tournament_selection
)
from src.simulator import evaluate_individual, schedule_muelles
from src.ga_utils import (
    local_search_on_routes, population_diversity, merge_routes_local_search
)

import pandas as pd
import numpy as np
import json
from copy import deepcopy
import random
import matplotlib.pyplot as plt

print("\n" + "="*80)
print("PASO 1: LIBRERÍAS IMPORTADAS")
print("="*80)

print("\nMódulos cargados correctamente:")
print("  - data_loader: Parsing de archivos .dat")
print("  - encoding: Codificación de soluciones y operadores GA")
print("  - simulator: Evaluación de rutas (costo + penalizaciones)")
print("  - ga_utils: Búsqueda local y refinamiento")
print("  - Utilidades: pandas, numpy, matplotlib")

print(f"\nSiguiente paso: Ejecuta el Paso 2 (Cargar datos)")
print("="*80)


PASO 1: LIBRERÍAS IMPORTADAS

Módulos cargados correctamente:
  - data_loader: Parsing de archivos .dat
  - encoding: Codificación de soluciones y operadores GA
  - simulator: Evaluación de rutas (costo + penalizaciones)
  - ga_utils: Búsqueda local y refinamiento
  - Utilidades: pandas, numpy, matplotlib

Siguiente paso: Ejecuta el Paso 2 (Cargar datos)


---

## <a name="paso-2"></a> Paso 2: Carga de Datos

Cargar la instancia del problema desde archivo AMPL (.dat). Modifica la ruta `dat_file` para usar otro archivo si es necesario.

In [141]:
# CONFIGURABLE: Cambiar esta ruta si tienes otro archivo .dat
dat_file = r'C:\Users\sjaim\Downloads\instancia_generada.dat'

In [142]:
# Parsear y construir la instancia
parsed = parse_ampl_dat(dat_file)
inst = build_instance(parsed)

print("\n" + "="*80)
print("PASO 2: INSTANCIA CARGADA DEL ARCHIVO")
print("="*80)

print(f"\nArchivo: {dat_file}")
print(f"  Nodos totales: {inst.n_nodes()} (1 depósito + {inst.n_nodes()-1} clientes)")
print(f"  Camiones disponibles: {len(inst.trucks)}")
print(f"  Clientes a visitar: {len([c for c in inst.clients.values() if c.escliente == 1])}")
print(f"  Estado: Listo para análisis")

print(f"\nSiguiente paso: Ejecuta el Paso 3 (Analizar instancia)")
print("="*80)


PASO 2: INSTANCIA CARGADA DEL ARCHIVO

Archivo: C:\Users\sjaim\Downloads\instancia_generada.dat
  Nodos totales: 21 (1 depósito + 20 clientes)
  Camiones disponibles: 1
  Clientes a visitar: 20
  Estado: Listo para análisis

Siguiente paso: Ejecuta el Paso 3 (Analizar instancia)


---

## <a name="paso-3"></a> Paso 3: Análisis de la Instancia

Examen detallado de los camiones disponibles, clientes a servir y parámetros de penalización.

In [143]:
# Analizar parámetros del problema

print("\n" + "="*70)
print("PASO 3: ANÁLISIS DE LA INSTANCIA")
print("="*70)

# Tabla de camiones
print("\nCAMIONES DISPONIBLES:")
print("-" * 70)
truck_data = []
for tid, truck in inst.trucks.items():
    tipo = "Por Hora" if truck.esHora else ("Franja 12h" if truck.esF12 else "Franja 6h")
    truck_data.append({
        'ID': tid,
        'Capacidad': f"{truck.Cap:.0f}",
        'Costo/Hora': f"${truck.CH:.1f}" if truck.esHora else "-",
        'Costo 6h': f"${truck.CF6:.1f}" if truck.esF6 else "-",
        'Costo 12h': f"${truck.CF12:.1f}" if truck.esF12 else "-",
        'Tipo': tipo
    })

df_trucks = pd.DataFrame(truck_data)
print(df_trucks.to_string(index=False))

# Tabla de clientes
print("\n\nCLIENTES A VISITAR:")
print("-" * 70)
client_data = []
for cid, client in inst.clients.items():
    if client.escliente == 1:
        client_data.append({
            'ID': cid,
            'Entrega': f"{client.DemE:.1f}",
            'Recogida': f"{client.DemR:.1f}",
            'Tiempo Servicio': f"{client.TS:.2f}h",
            'Min Llegada': f"{client.MinDC:.1f}",
            'Max Llegada': f"{client.MaxDC:.1f}",
            'Crítico': 'Sí' if client.escritico == 1 else 'No'
        })

df_clients = pd.DataFrame(client_data)
print(df_clients.to_string(index=False))

# Parámetros de penalización
print("\n\nPARAMETROS DE PENALIZACIÓN:")
print("-" * 70)
penalties = {
    'Llegar antes en cliente crítico': inst.params.get('pcmin_c', 10),
    'Llegar después en cliente crítico': inst.params.get('pcmax_c', 15),
    'Llegar antes en cliente no crítico': inst.params.get('pcmin_nc', 5),
    'Llegar después en cliente no crítico': inst.params.get('pcmax_nc', 8),
    'Regresar después de 18:00': inst.params.get('preg', 20),
    'Por HORA de espera': inst.params.get('pw', 1000),
}

for penalidad, valor in penalties.items():
    print(f"  {penalidad}: {valor:,.0f} pts")


PASO 3: ANÁLISIS DE LA INSTANCIA

CAMIONES DISPONIBLES:
----------------------------------------------------------------------
 ID Capacidad Costo/Hora Costo 6h Costo 12h      Tipo
  1        95          -   $300.0         - Franja 6h


CLIENTES A VISITAR:
----------------------------------------------------------------------
 ID Entrega Recogida Tiempo Servicio Min Llegada Max Llegada Crítico
  1    10.0      1.0           0.11h         9.0        16.0      Sí
  2     5.0      3.0           0.22h        12.0        20.0      Sí
  3    12.0      2.0           0.19h        11.0        15.0      No
  4    25.0      5.0           0.18h         6.0        12.0      No
  5    21.0      6.0           0.26h         6.0        13.0      No
  6    19.0      5.0           0.20h         8.0        11.0      No
  7    21.0      0.0           0.19h         6.0        10.0      No
  8     5.0      1.0           0.27h        10.0        13.0      No
  9    20.0      0.0           0.23h        12.0  

---

## <a name="paso-4"></a> Paso 4: Operadores Genéticos

Demostración de los operadores genéticos principales utilizados en el algoritmo.

In [31]:
# Paso 4: Entender la codificación de soluciones

print("\n" + "="*70)
print("PASO 4: CODIFICACIÓN DE SOLUCIONES")
print("="*70)

print("\nCómo representamos una solución:")
print("-" * 70)
print("""
Una solución es un VECTOR donde:
  - Números = IDs de clientes a visitar
  - Ceros = Separadores entre rutas (uno por camión)
  
Ejemplo:
  Vector: [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
  
  Interpretación (4 camiones):
    - Camión 1: Ruta [1, 5, 3]        (3 clientes)
    - Camión 2: Ruta [2, 8, 6]        (3 clientes)
    - Camión 3: Ruta [10, 9, 4, 7]    (4 clientes)
    - Camión 4: Ruta []               (vacío, no se usa)
""")

# Ejemplo visual
print("\nEjemplo de codificación:")
print("-" * 70)
vector_ejemplo = [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
print(f"Vector: {vector_ejemplo}")
print(f"Rutas decodificadas: {decode_vector(vector_ejemplo)}")
print(f"\nCada ruta será evaluada en términos de:")
print(f"  - Tiempo total de la ruta")
print(f"  - Costo (contrato del camión)")
print(f"  - Penalizaciones (ventanas de tiempo, esperas)")


PASO 4: CODIFICACIÓN DE SOLUCIONES

Cómo representamos una solución:
----------------------------------------------------------------------

Una solución es un VECTOR donde:
  - Números = IDs de clientes a visitar
  - Ceros = Separadores entre rutas (uno por camión)

Ejemplo:
  Vector: [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]

  Interpretación (4 camiones):
    - Camión 1: Ruta [1, 5, 3]        (3 clientes)
    - Camión 2: Ruta [2, 8, 6]        (3 clientes)
    - Camión 3: Ruta [10, 9, 4, 7]    (4 clientes)
    - Camión 4: Ruta []               (vacío, no se usa)


Ejemplo de codificación:
----------------------------------------------------------------------
Vector: [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
Rutas decodificadas: [[1, 5, 3], [2, 8, 6], [10, 9, 4, 7]]

Cada ruta será evaluada en términos de:
  - Tiempo total de la ruta
  - Costo (contrato del camión)
  - Penalizaciones (ventanas de tiempo, esperas)


---

### Operadores Genéticos Utilizados

El algoritmo emplea tres operadores principales para explorar el espacio de soluciones:

**1. Cruce (Crossover) - Route-Based Crossover (RBX)**
- Selecciona segmentos completos de dos soluciones padre
- Combina rutas preservando la estructura del problema
- Herada características favorables de ambos progenitores

**2. Mutación - SWAP (Intercambio)**
- Intercambia posiciones de dos clientes en el vector de solución
- Explora el vecindario de una solución
- Probabilidad de aplicación: controlada por parámetro `MUTATION_RATE`

**3. Mutación - INSERT (Inserción)**
- Extrae un cliente de su posición y lo inserta en otra
- Permite reorganización de rutas sin crear duplicados
- Operador alternativo para diversificación

In [32]:
# Ejemplo 1: CRUCE (Route-Based Crossover - RBX)

print("\n" + "="*70)
print("OPERADOR 1: CRUCE (Route-Based Crossover)")
print("="*70)

padre_A = [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
padre_B = [0, 4, 6, 3, 0, 2, 9, 8, 0, 1, 7, 10, 5, 0]

print(f"\nPadre A: {padre_A}")
print(f"   Rutas: {decode_vector(padre_A)}")

print(f"\nPadre B: {padre_B}")
print(f"   Rutas: {decode_vector(padre_B)}")

hijo = route_based_crossover(padre_A, padre_B)

print(f"\nHijo (RBX): {hijo}")
print(f"   Rutas: {decode_vector(hijo)}")

print("\nQue pasó:")
print("   El hijo heredó rutas completas del Padre A y B")
print("   Esto mantiene la estructura del problema intacta")


OPERADOR 1: CRUCE (Route-Based Crossover)

Padre A: [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
   Rutas: [[1, 5, 3], [2, 8, 6], [10, 9, 4, 7]]

Padre B: [0, 4, 6, 3, 0, 2, 9, 8, 0, 1, 7, 10, 5, 0]
   Rutas: [[4, 6, 3], [2, 9, 8], [1, 7, 10, 5]]

Hijo (RBX): [0, 1, 5, 3, 0, 4, 6, 2, 9, 0, 8, 7, 10, 0]
   Rutas: [[1, 5, 3], [4, 6, 2, 9], [8, 7, 10]]

Que pasó:
   El hijo heredó rutas completas del Padre A y B
   Esto mantiene la estructura del problema intacta


---

### Ejemplos de Operadores en Acción

In [33]:
# Ejemplo 2: MUTACIÓN - SWAP (Intercambio)

print("\n" + "="*70)
print("OPERADOR 2A: MUTACIÓN SWAP (Intercambiar dos clientes)")
print("="*70)

original = [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
print(f"\nVector original: {original}")
print(f"   Rutas: {decode_vector(original)}")

mutado = swap_mutation(original)
print(f"\nVector mutado: {mutado}")
print(f"   Rutas: {decode_vector(mutado)}")

print("\nQué pasó:")
print("   Se intercambiaron dos clientes de posición")
print("   Esto cambia el orden de visita (puede mejorar o empeorar)")
print("   La búsqueda local después corregirá el orden si es malo")


OPERADOR 2A: MUTACIÓN SWAP (Intercambiar dos clientes)

Vector original: [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
   Rutas: [[1, 5, 3], [2, 8, 6], [10, 9, 4, 7]]

Vector mutado: [0, 4, 5, 3, 2, 0, 8, 6, 10, 0, 9, 1, 7, 0]
   Rutas: [[4, 5, 3, 2], [8, 6, 10], [9, 1, 7]]

Qué pasó:
   Se intercambiaron dos clientes de posición
   Esto cambia el orden de visita (puede mejorar o empeorar)
   La búsqueda local después corregirá el orden si es malo


---

### 🔄 Mutación INSERT

In [34]:
# Ejemplo 3: MUTACIÓN - INSERT (Reinsertar cliente)

print("\n" + "="*70)
print("OPERADOR 2B: MUTACIÓN INSERT (Mover cliente a otra posición)")
print("="*70)

original = [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
print(f"\nVector original: {original}")
print(f"   Rutas: {decode_vector(original)}")

mutado = insert_mutation(original)
print(f"\nVector mutado: {mutado}")
print(f"   Rutas: {decode_vector(mutado)}")

print("\nQué pasó:")
print("   Se tomó un cliente y se insertó en otra posición")
print("   Puede cambiar un cliente entre rutas o solo reordenar")
print("   Permite explorar asignaciones completamente distintas")


OPERADOR 2B: MUTACIÓN INSERT (Mover cliente a otra posición)

Vector original: [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
   Rutas: [[1, 5, 3], [2, 8, 6], [10, 9, 4, 7]]

Vector mutado: [0, 3, 1, 5, 2, 0, 8, 6, 10, 0, 9, 4, 7, 0]
   Rutas: [[3, 1, 5, 2], [8, 6, 10], [9, 4, 7]]

Qué pasó:
   Se tomó un cliente y se insertó en otra posición
   Puede cambiar un cliente entre rutas o solo reordenar
   Permite explorar asignaciones completamente distintas


---

## <a name="paso-5"></a> Paso 5: Configuración del Algoritmo Genético

Definición de parámetros que controlan la ejecución del algoritmo.

In [35]:
print("OPERADOR: MUTACIÓN SWAP")
print("="*60)

original = [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
print(f"\nVector original: {original}")
print(f"Rutas: {decode_vector(original)}")

mutado = swap_mutation(original)
print(f"\nVector mutado: {mutado}")
print(f"Rutas: {decode_vector(mutado)}")

print("\nQué pasó:")
print("  Se intercambiaron dos clientes de posición.")
print("  Esto permite explorar diferentes órdenes de visita.")

OPERADOR: MUTACIÓN SWAP

Vector original: [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
Rutas: [[1, 5, 3], [2, 8, 6], [10, 9, 4, 7]]

Vector mutado: [0, 6, 5, 3, 2, 0, 8, 1, 10, 0, 9, 4, 7, 0]
Rutas: [[6, 5, 3, 2], [8, 1, 10], [9, 4, 7]]

Qué pasó:
  Se intercambiaron dos clientes de posición.
  Esto permite explorar diferentes órdenes de visita.


### Mutación - Inserción

Extrae un cliente de su posición actual e inserta en otra ubicación, permitiendo reorganización de rutas.

In [36]:
print("OPERADOR: MUTACIÓN INSERT")
print("="*60)

original = [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
print(f"\nVector original: {original}")
print(f"Rutas: {decode_vector(original)}")

mutado = insert_mutation(original)
print(f"\nVector mutado: {mutado}")
print(f"Rutas: {decode_vector(mutado)}")

print("\nQué pasó:")
print("  Se tomó un cliente y se insertó en otra posición.")
print("  Permite cambiar el orden de visita y potencialmente mejorar la ruta.")

OPERADOR: MUTACIÓN INSERT

Vector original: [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
Rutas: [[1, 5, 3], [2, 8, 6], [10, 9, 4, 7]]

Vector mutado: [0, 5, 3, 2, 8, 0, 6, 10, 1, 0, 9, 4, 7, 0]
Rutas: [[5, 3, 2, 8], [6, 10, 1], [9, 4, 7]]

Qué pasó:
  Se tomó un cliente y se insertó en otra posición.
  Permite cambiar el orden de visita y potencialmente mejorar la ruta.


In [37]:
# PASO 5: Configuración del Algoritmo Genético

print("\n" + "="*70)
print("PASO 5: CONFIGURACIÓN DEL GA")
print("="*70)

# Parámetros principales del GA
POPSIZE = 300        # Número de soluciones en cada generación
GENS = 1200          # Número de generaciones a ejecutar
SEED = 42            # Semilla para reproducibilidad
MUTATION_RATE = 0.40 # Probabilidad de mutar
ROUTE_PENALTY = 500  # Penalización por rutas >12h

# Inicializar seed
random.seed(SEED)
np.random.seed(SEED)

print(f"\nParámetros del GA:")
print(f"  Tamaño de población: {POPSIZE} individuos")
print(f"  Generaciones: {GENS}")
print(f"  Semilla aleatoria: {SEED}")
print(f"  Tasa de mutación: {MUTATION_RATE*100:.0f}%")
print(f"  Penalización rutas largas: {ROUTE_PENALTY} pts/hora extra")

print(f"\nOperadores:")
print(f"  Selección: Torneo (k=5)")
print(f"  Cruce: RBX (Route-Based Crossover)")
print(f"  Mutación: SWAP 70% + INSERT 20% + Agresiva 10%")
print(f"  Búsqueda local: 50% (adaptativa hasta 80%)")
print(f"  Elitismo: Mantener 10% mejores")

print(f"\nCriterios de parada:")
print(f"  150 generaciones sin mejora")
print(f"  O {GENS} generaciones alcanzadas")


PASO 5: CONFIGURACIÓN DEL GA

Parámetros del GA:
  Tamaño de población: 300 individuos
  Generaciones: 1200
  Semilla aleatoria: 42
  Tasa de mutación: 40%
  Penalización rutas largas: 500 pts/hora extra

Operadores:
  Selección: Torneo (k=5)
  Cruce: RBX (Route-Based Crossover)
  Mutación: SWAP 70% + INSERT 20% + Agresiva 10%
  Búsqueda local: 50% (adaptativa hasta 80%)
  Elitismo: Mantener 10% mejores

Criterios de parada:
  150 generaciones sin mejora
  O 1200 generaciones alcanzadas


In [38]:
# Crear población inicial aleatoria

def random_initial_population(inst, pop_size=200, seed=42):
    """
    Genera una población inicial con asignaciones aleatorias de clientes a camiones.
    
    Args:
        inst: Instancia del problema
        pop_size: Número de individuos a generar
        seed: Semilla para reproducibilidad
        
    Returns:
        Lista de vectores codificados (soluciones)
    """
    rng = random.Random(seed)
    client_ids = [nid for nid, c in inst.clients.items() if c.escliente == 1]
    R = len(inst.trucks)
    population = []
    
    for _ in range(pop_size):
        # Permutar aleatoriamente los clientes
        perm = client_ids[:]
        rng.shuffle(perm)
        
        # Distribuir clientes entre camiones
        routes = []
        base = len(perm) // R
        rem = len(perm) % R
        idx = 0
        
        for r in range(R):
            size = base + (1 if r < rem else 0)
            routes.append(perm[idx:idx+size])
            idx += size
        
        population.append(encode_routes(routes))
    
    return population

# Crear población inicial
population = random_initial_population(inst, pop_size=POPSIZE, seed=SEED)

print("\n" + "="*70)
print("PASO 5B: POBLACIÓN INICIAL CREADA")
print("="*70)
print(f"\nPoblación inicial generada: {len(population)} individuos")
print(f"  Cada individuo es una asignación aleatoria de clientes a camiones")
print(f"  Ejemplo de 3 individuos:")
for i in range(min(3, len(population))):
    print(f"    Individual {i+1}: {population[i][:20]}..." if len(population[i]) > 20 else f"    Individual {i+1}: {population[i]}")


PASO 5B: POBLACIÓN INICIAL CREADA

Población inicial generada: 300 individuos
  Cada individuo es una asignación aleatoria de clientes a camiones
  Ejemplo de 3 individuos:
    Individual 1: [0, 8, 6, 3, 9, 0, 10, 7, 12, 4, 0, 5, 1, 2, 11, 0]
    Individual 2: [0, 6, 8, 3, 10, 0, 11, 7, 5, 9, 0, 4, 2, 12, 1, 0]
    Individual 3: [0, 9, 11, 6, 3, 0, 12, 2, 7, 1, 0, 5, 10, 8, 4, 0]


---

## <a name="paso-6"></a> Paso 6: Ejecución del Algoritmo Genético

### ¿Cómo funciona el Algoritmo Genético?

Esta combinación equilibra **explotación** (mejorar buenas soluciones) y **exploración** (descubrir nuevas regiones).

El GA es un método de optimización inspirado en la evolución natural que:

- **40% Aleatorias**: Diversidad para explorar zonas no convencionales del espacio de búsqueda

1. **Inicializa** una población de soluciones candidatas  

2. **Evalúa** cada solución según la función objetivo  - Variantes aleatorias de construcción

3. **Selecciona** las mejores soluciones para reproducirse  - Ordenamiento por urgencia (ventanas de tiempo)

4. **Cruza** soluciones para crear nuevas combinaciones  - Ordenamiento por carga (balanceo de capacidad)

5. **Muta** algunas soluciones para explorar nuevas posibilidades- **60% Heurísticas greedy**: Soluciones de buena calidad inicial que guían la búsqueda

6. **Reemplaza** las soluciones más débiles con las nuevas generadas

7. **Repite** el proceso durante múltiples generaciones**¿Por qué combinar heurísticas con aleatoriedad?**


### Estrategia de Inicialización

### 6.1 Inicialización de Población

Se crea una población inicial de 100 soluciones usando:

- Heurísticas constructivas inteligentes- Evaluación de calidad de cada solución
- Generación aleatoria para diversidad

### 6.2 Evolución Genética

El algoritmo evoluciona las soluciones mediante ciclos iterativos de mejora.

---

## <a name="paso-4"></a> Paso 4: Configuración del Algoritmo Genético

### Parámetros del GA



La configuración del algoritmo determina su comportamiento y eficiencia. Aquí se explica cada parámetro y su función:  - Mantiene capacidad de exploración

  - Evita convergencia prematura

#### Parámetros de Población- **Regeneración (30%)**: Reemplaza 30% de peores soluciones con nuevas

- **Tamaño de población (100)**: Número de soluciones que evolucionan simultáneamente- **Umbral (0.80)**: Si la diversidad cae bajo 80%, se activa regeneración

  - *Más individuos* → Mayor diversidad pero más tiempo de cómputo- **Intervalo de revisión (50)**: Verifica diversidad cada 50 generaciones

  - *Menos individuos* → Convergencia más rápida pero mayor riesgo de estancamiento#### Control de Diversidad



#### Parámetros de Evolución  - Se intensifica en las últimas generaciones

- **Generaciones (500)**: Número máximo de ciclos evolutivos  - Mejora soluciones mediante movimientos locales

  - Cada generación aplica selección, cruce, mutación y reemplazo- **Búsqueda local (30%)**: Optimización adicional en 30% de las rutas

  - El algoritmo puede parar antes si encuentra una buena solución

  - Garantiza que nunca se pierdan las mejores soluciones

- **Parada anticipada (60)**: Detiene si no hay mejora en 60 generaciones consecutivas- **Elitismo (2)**: Los 2 mejores pasan intactos a la siguiente generación

  - Evita cómputo innecesario cuando el algoritmo ha convergido#### Mecanismos de Calidad



#### Operadores Genéticos  - REASSIGN (10%): Mueve cadenas entre rutas

- **Selección por torneo (k=3)**: Compara 3 soluciones y elige la mejor  - INSERT (20%): Reubica un cliente

  - Mayor k → Presión selectiva más fuerte  - SWAP (70%): Intercambia dos clientes

  - Menor k → Mayor diversidad- **Mutación (10%)**: Probabilidad de modificar una solución



- **Cruce RBX (85%)**: Probabilidad de combinar dos soluciones padres  - Mantiene estructura de las buenas soluciones
  - Recombina rutas completas de los padres

### Pesos de Priorización de Muelles (w1-w5)

**¿Por qué priorizar rutas?**

Cuando múltiples camiones deben salir, algunos muelles de carga están ocupados. Es necesario decidir qué camión sale primero. Los pesos determinan la importancia de cada criterio:

- **w1 (0.40) - Urgencia de ventanas**: Prioriza rutas con clientes de ventanas estrictas
- **w2 (0.30) - Duración de ruta**: Rutas largas necesitan salir temprano
- **w3 (0.15) - Clientes críticos**: Rutas con más clientes importantes
- **w4 (0.10) - Riesgo de atraso**: Probabilidad de llegar tarde
- **w5 (0.05) - Sensibilidad al tráfico**: Rutas en horarios de congestión


In [144]:
# ============================================================================
# PASO 7.1: CONFIGURACIÓN COMPLETA DEL ALGORITMO GENÉTICO
# ============================================================================

print("\n" + "="*80)
print("PASO 7: ALGORITMO GENÉTICO ESTRUCTURADO Y OPTIMIZADO")
print("="*80)

# Parámetros del GA
GA_PARAMS = {
    'popsize': 100,                # Tamaño de población
    'gens': 500,                   # Número máximo de generaciones
    'seed': 42,                    # Semilla aleatoria
    'k_tournament': 3,             # Tamaño de torneo para selección
    'prob_cruce': 0.85,            # Probabilidad de aplicar RBX
    'prob_mutacion': 0.10,         # Probabilidad de mutar
    'elite_size': 2,               # Número de élites
    'local_search_fraction': 0.30, # Fracción de rutas para búsqueda local
    'diversity_interval': 50,      # Cada N generaciones revisar diversidad
    'diversity_threshold': 0.80,   # Umbral de diversidad
    'diversity_regen': 0.30,       # Fracción para regenerar si diversidad baja
    'early_stop_gens': 60,         # Parada si N generaciones sin mejora
}

# Pesos para ordenamiento de muelles (w1-w5)
WEIGHTS = {
    'w1': 0.40,  # Urgencia por ventanas de tiempo
    'w2': 0.30,  # Duración estimada de la ruta
    'w3': 0.15,  # Cantidad de clientes críticos
    'w4': 0.10,  # Riesgo de atraso
    'w5': 0.05,  # Sensibilidad al tráfico
}

# **AJUSTE CRÍTICO: Multiplicador de costo de contrato**
COST_MULTIPLIER = 1  # Penalidad por uso de contrato de camión

print("\nConfiguración del GA:")
print(f"  Tamaño población:        {GA_PARAMS['popsize']}")
print(f"  Generaciones:            {GA_PARAMS['gens']}")
print(f"  Selección (torneo k):    {GA_PARAMS['k_tournament']}")
print(f"  Cruce RBX:               {GA_PARAMS['prob_cruce']*100:.0f}%")
print(f"  Mutación:                {GA_PARAMS['prob_mutacion']*100:.0f}%")
print(f"  Elitismo:                {GA_PARAMS['elite_size']}")
print(f"  Búsqueda local:          {GA_PARAMS['local_search_fraction']*100:.0f}%")
print(f"  Parada anticipada:       {GA_PARAMS['early_stop_gens']} generaciones sin mejora")

print("\nPesos para asignación de muelles (w1-w5):")
for k, v in WEIGHTS.items():
    print(f"  {k}: {v}")

print(f"\n*** MULTIPLICADOR DE COSTO DE CONTRATO: {COST_MULTIPLIER} ***")



PASO 7: ALGORITMO GENÉTICO ESTRUCTURADO Y OPTIMIZADO

Configuración del GA:
  Tamaño población:        100
  Generaciones:            500
  Selección (torneo k):    3
  Cruce RBX:               85%
  Mutación:                10%
  Elitismo:                2
  Búsqueda local:          30%
  Parada anticipada:       60 generaciones sin mejora

Pesos para asignación de muelles (w1-w5):
  w1: 0.4
  w2: 0.3
  w3: 0.15
  w4: 0.1
  w5: 0.05

*** MULTIPLICADOR DE COSTO DE CONTRATO: 1 ***


## <a name="paso-5"></a> Paso 5: Operadores y Funciones del GA

### ¿Qué son los operadores genéticos?

Los operadores genéticos son las "herramientas" que usa el algoritmo para crear y mejorar soluciones:

#### 1. Operadores de Cruce
- **Route-Based Crossover (RBX)**: Combina rutas completas de dos padres

  - Mantiene la estructura de rutas exitosas

  - Evita romper secuencias buenas de clientes**Objetivo**: Minimizar Z encontrando el mejor balance entre usar menos camiones y cumplir restricciones.



#### 2. Operadores de Mutación- Tiempo de espera acumulado

- **SWAP**: Intercambia posiciones de dos clientes- Regreso tardío al depósito (después de 18:00)

  - Explora diferentes ordenamientos- Llegadas tempranas/tardías a clientes no críticos  

- **INSERT**: Reubica un cliente a otra posición- Llegadas tempranas/tardías a clientes críticos

  - Ajusta secuencias de visita**Penalizaciones:**

- **REASSIGN**: Mueve cadenas de clientes entre rutas

  - Redistribuye carga entre camiones- Camiones por franja: `CF6` o `CF12` (fijo)

- Camiones por hora: `CH × tiempo_total`

#### 3. Búsqueda Local**Costos de Contratos:**

- **2-opt**: Invierte segmentos de ruta para eliminar cruces

- **Reubicación**: Mueve clientes entre rutas si mejora la solución```

- **Merge routes**: Combina rutas cuando es posibleZ = Costo_Contratos + Penalizaciones

```

#### 4. Reparación de Factibilidad

La función objetivo (Z) calcula el costo total:

Cuando una solución viola restricciones, se repara verificando:

- **Capacidad**: No exceder la carga máxima del camión**¿Cómo se evalúa una solución?**

- **Tiempo máximo**: Rutas no mayores a 12 horas

- **Ventanas de tiempo**: Visitar clientes en horarios permitidos### Función Objetivo

- **Almuerzo**: Obligatorio después de las 14:00
- **Muelles**: Disponibilidad de muelles de carga

In [147]:
# ============================================================================
# PASO 7.2: FUNCIONES AUXILIARES DEL ALGORITMO GENÉTICO
# ============================================================================

def mutacion_punto_corte(parent_a, parent_b, rng=random):
    """
    Mutación especializada: Punto de corte + Rellenar
    1. Seleccionar un punto de corte en el padre A
    2. Copiar ese segmento del padre A
    3. Rellenar con clientes faltantes del padre B (en orden)
    4. Mantener cantidad original de separadores (0)
    """
    routes_a = decode_vector(parent_a)
    routes_b = decode_vector(parent_b)
    
    num_routes = len(routes_a)
    cut_point = rng.randint(0, num_routes - 1)
    
    new_routes = [routes_a[i][:] for i in range(cut_point)]
    used = set(cid for r in new_routes for cid in r)
    
    for rb in routes_b:
        nr = [cid for cid in rb if cid not in used]
        used.update(nr)
        if nr or len(new_routes) < num_routes:
            new_routes.append(nr)
    while len(new_routes) < num_routes:
        new_routes.append([])
    new_routes = new_routes[:num_routes]
    return encode_routes(new_routes)


def verificar_factibilidad(individual, inst):
    """Verifica capacidad, tiempo máximo, ventanas, muelles y almuerzo > 14:00"""
    res = evaluate_individual(individual, inst)
    violations = { 'capacity': 0, 'max_time': 0, 'time_windows': 0, 'muelles': 0, 'almuerzo': 0 }
    for _, det in res['details'].items():
        if det.get('cap_violation', 0) > 1e-6: violations['capacity'] += 1
        if det.get('TT', 0) > 12.1: violations['max_time'] += 1
        # time windows covered via penalties in res
    return { 'feasible': sum(violations.values()) == 0, 'violations': violations, 'eval': res }


def calcular_prioridad_muelles(routes, inst, weights):
    """Calcula prioridad por w1-w5 y retorna [(idx, score)] ordenados desc."""
    priorities = []
    for idx, route in enumerate(routes):
        if not route:
            priorities.append((idx, -1e10)); continue
        score = 0.0
        min_window = min((inst.clients[cid].MinDC for cid in route), default=24.0)
        w1_val = 1.0 - (min_window / 24.0)
        score += weights['w1'] * w1_val
        est_time = len(route) * 0.5
        w2_val = min(1.0, est_time / 12.0)
        score += weights['w2'] * w2_val
        critical = sum(1 for cid in route if inst.clients[cid].escritico == 1)
        w3_val = critical / max(1, len(route))
        score += weights['w3'] * w3_val
        total_width = sum(inst.clients[cid].MaxDC - inst.clients[cid].MinDC for cid in route)
        avg_width = total_width / len(route) if route else 24.0
        w4_val = 1.0 - (avg_width / 24.0)
        score += weights['w4'] * w4_val
        pico = sum(1 for cid in route if 8.0 <= inst.clients[cid].MinDC <= 12.0)
        w5_val = pico / max(1, len(route))
        score += weights['w5'] * w5_val
        priorities.append((idx, score))
    priorities.sort(key=lambda x: x[1], reverse=True)
    return priorities


def evaluar_poblacion(population, inst):
    # Usa la versión con multiplicador de costo de contrato y extrae solo el valor Z
    return [evaluate_individual_with_multiplier(ind, inst, cost_multiplier=COST_MULTIPLIER)['Z'] for ind in population]


def diversidad_poblacion(population):
    if len(population) < 2: return 1.0
    distances = []
    for i in range(min(10, len(population))):
        for j in range(i+1, min(10, len(population))):
            distances.append(sum(1 for a, b in zip(population[i], population[j]) if a != b))
    if not distances: return 1.0
    max_dist = len(population[0]); avg_distance = np.mean(distances)
    return min(1.0, avg_distance / max_dist if max_dist > 0 else 1.0)


def greedy_initial_population(inst, pop_size=100, num_variants=3, seed=42):
    random.seed(seed); np.random.seed(seed)
    population = []
    clients = [c for c in inst.clients.values() if c.escliente == 1]
    truck_ids = sorted(inst.trucks.keys())
    trucks_list = [inst.trucks[tid] for tid in truck_ids]
    num_trucks = len(trucks_list)
    def current_load(routes, t):
        return sum(inst.clients[x].DemE + inst.clients[x].DemR for x in routes[t])
    def can_add(routes, t, cid):
        return current_load(routes, t) + inst.clients[cid].DemE + inst.clients[cid].DemR <= trucks_list[t].Cap
    def construct_greedy_balanced():
        routes = [[] for _ in range(num_trucks)]
        unvisited = sorted([c.id for c in clients], key=lambda cid: inst.clients[cid].DemE + inst.clients[cid].DemR, reverse=True)
        for cid in unvisited:
            t = min(range(num_trucks), key=lambda k: current_load(routes, k))
            if can_add(routes, t, cid): routes[t].append(cid)
        return encode_routes(routes)
    def construct_greedy_urgency():
        routes = [[] for _ in range(num_trucks)]
        ordered = sorted([c.id for c in clients], key=lambda cid: inst.clients[cid].MinDC)
        for cid in ordered:
            t = min(range(num_trucks), key=lambda k: current_load(routes, k))
            if can_add(routes, t, cid): routes[t].append(cid)
        return encode_routes(routes)
    def construct_random():
        routes = [[] for _ in range(num_trucks)]
        unvisited = [c.id for c in clients]; random.shuffle(unvisited)
        for cid in unvisited:
            t = min(range(num_trucks), key=lambda k: current_load(routes, k))
            if can_add(routes, t, cid): routes[t].append(cid)
        return encode_routes(routes)
    constructors = [construct_greedy_balanced, construct_greedy_urgency, construct_random]
    for i in range(pop_size): population.append(constructors[i % len(constructors)]())
    return population

# --- Nuevos operadores y reparadores ---

def reassign_chain_mutation(individual, inst, rng=random):
    routes = decode_vector(individual)
    if sum(1 for r in routes if r) < 2: return individual
    src = rng.choice([i for i, r in enumerate(routes) if len(r) >= 2])
    dst = rng.choice([i for i, r in enumerate(routes) if i != src])
    rsrc = routes[src]
    i = rng.randint(0, len(rsrc)-2); j = rng.randint(i+1, len(rsrc)-1)
    chain = rsrc[i:j+1]
    cap_dst = sum(inst.clients[cid].DemE + inst.clients[cid].DemR for cid in routes[dst])
    chain_dem = sum(inst.clients[cid].DemE + inst.clients[cid].DemR for cid in chain)
    truck_ids = sorted(inst.trucks.keys()); trucks_list = [inst.trucks[tid] for tid in truck_ids]
    if cap_dst + chain_dem > trucks_list[dst].Cap: return individual
    routes[src] = rsrc[:i] + rsrc[j+1:]
    insert_pos = rng.randint(0, len(routes[dst]))
    routes[dst] = routes[dst][:insert_pos] + chain + routes[dst][insert_pos:]
    return encode_routes(routes)


def repair_feasibility(individual, inst):
    routes = decode_vector(individual)
    changed = False
    for r_idx, route in enumerate(routes):
        if len(route) >= 3:
            for i in range(len(route)-1):
                for j in range(i+2, min(i+5, len(route))):
                    new_route = route[:i+1] + route[i+1:j+1][::-1] + route[j+1:]
                    tmp = encode_routes([routes[k] if k != r_idx else new_route for k in range(len(routes))])
                    if evaluate_individual(tmp, inst)['Z'] < evaluate_individual(encode_routes(routes), inst)['Z']:
                        routes[r_idx] = new_route; changed = True; break
                if changed: break
        if changed: break
    if changed: return encode_routes(routes)
    longest = max(range(len(routes)), key=lambda k: len(routes[k]) if routes[k] else -1)
    if not routes[longest]: return individual
    cid = routes[longest].pop()
    truck_ids = sorted(inst.trucks.keys()); trucks_list = [inst.trucks[tid] for tid in truck_ids]
    best_t = min(range(len(routes)), key=lambda k: sum(inst.clients[x].DemE + inst.clients[x].DemR for x in routes[k]))
    load = sum(inst.clients[x].DemE + inst.clients[x].DemR for x in routes[best_t])
    if load + inst.clients[cid].DemE + inst.clients[cid].DemR <= trucks_list[best_t].Cap:
        routes[best_t].append(cid); return encode_routes(routes)
    routes[longest].append(cid); return individual


def prioritize_routes_by_weights(individual, inst, weights):
    routes = decode_vector(individual)
    pr = calcular_prioridad_muelles(routes, inst, weights)
    ordered_routes = [routes[i] for i, _ in pr]
    return encode_routes(ordered_routes)

print("✓ Funciones auxiliares del GA definidas correctamente")

✓ Funciones auxiliares del GA definidas correctamente


### Representación de Soluciones

**¿Cómo se codifica una solución?**


Cada solución es un vector que representa las rutas de todos los camiones:7. **Devolver**: Valor de función objetivo Z 

6. **Calcular**: Sumar costos de contratos y penalizaciones

```5. **Verificar**: Detectar violaciones de restricciones

[0, cliente_1, cliente_2, 0, cliente_3, cliente_4, cliente_5, 0]4. **Simular**: Calcular tiempos de llegada a cada cliente

```3. **Programar**: Asignar muelles y horarios de salida

2. **Priorizar**: Ordenar rutas por importancia (w1-w5)

- El `0` representa el depósito (punto de inicio/fin)1. **Decodificar**: Convertir vector a lista de rutas

- Los números entre `0`s forman la ruta de un camión

- Ejemplo: Camión 1 visita [1,2], Camión 2 visita [3,4,5]Cada solución se evalúa siguiendo estos pasos:



**Ventajas de esta codificación:**### Proceso de Evaluación

- Fácil de manipular con operadores genéticos

- Mantiene clara la separación entre rutas- Compatible con cualquier número de camiones

In [148]:
# ============================================================================
# PASO 7.3: INICIALIZACIÓN DE LA POBLACIÓN
# ============================================================================

# Reinicializar seed
random.seed(GA_PARAMS['seed'])
np.random.seed(GA_PARAMS['seed'])

# Crear población inicial: 60% GREEDY + 40% ALEATORIA
print(f"\nCreando población inicial de {GA_PARAMS['popsize']} individuos...")
greedy_size = int(GA_PARAMS['popsize'] * 0.6)
random_size = GA_PARAMS['popsize'] - greedy_size

# Generar población greedy
greedy_pop = greedy_initial_population(inst, pop_size=greedy_size, num_variants=3, seed=GA_PARAMS['seed'])
print(f"  ✓ {greedy_size} individuos con heurística greedy")

# Generar población aleatoria
random_pop = random_initial_population(inst, pop_size=random_size, seed=GA_PARAMS['seed']+1)
print(f"  ✓ {random_size} individuos aleatorios")

# Combinar
population = greedy_pop + random_pop

# Evaluar población inicial
fitness = evaluar_poblacion(population, inst)

# Encontrar el mejor inicial
best_idx = min(range(len(population)), key=lambda i: fitness[i])
best = deepcopy(population[best_idx])
best_score = fitness[best_idx]

print(f"✓ Población inicial creada (60% greedy + 40% aleatoria)")
print(f"\nESTADÍSTICAS GENERACIÓN 0:")
print(f"  Mejor Z:    ${min(fitness):.2f}")
print(f"  Peor Z:     ${max(fitness):.2f}")
print(f"  Promedio:   ${np.mean(fitness):.2f}")
print(f"  Desv. Est:  ${np.std(fitness):.2f}")

# Inicializar historial
history_best = [best_score]
history_avg = [np.mean(fitness)]
history_div = [diversidad_poblacion(population)]

# Contador de generaciones sin mejora
gens_since_improve = 0


Creando población inicial de 100 individuos...
  ✓ 60 individuos con heurística greedy
  ✓ 40 individuos aleatorios
✓ Población inicial creada (60% greedy + 40% aleatoria)

ESTADÍSTICAS GENERACIÓN 0:
  Mejor Z:    $470.60
  Peor Z:     $1252.48
  Promedio:   $711.65
  Desv. Est:  $247.35


### Ciclo Evolutivo del GA

**¿Qué ocurre en cada generación?**

El algoritmo repite el siguiente proceso hasta convergencia:

```
Para cada generación:
  1. Guardar élites (2 mejores soluciones)
  
  2. Para crear cada nuevo hijo:

     a) Seleccionar 2 padres por torneo (k=3)

     b) Aplicar cruce RBX con 85% de probabilidad- Número de generaciones sin mejora

     c) Aplicar mutación con 10% de probabilidad- Medida de diversidad

     d) Aplicar búsqueda local con 30% de probabilidad- Promedio de calidad de la población

     e) Ordenar rutas por prioridad (w1-w5)- Mejor solución de cada generación

     f) Reparar si es infactible**Salidas del proceso:**

  

  3. Reemplazar población con nuevos hijos + élites```

       - Parar (convergencia alcanzada)

  4. Evaluar toda la población  6. Si no hay mejora en 60 generaciones:

    

  5. Si es generación múltiplo de 50:     - Si diversidad < 80%: regenerar 30% de población
     - Calcular diversidad

In [149]:
# ============================================================================
# PASO 7.4: BUCLE PRINCIPAL DEL ALGORITMO GENÉTICO
# ============================================================================

print("\n" + "="*80)
print("EJECUTANDO ALGORITMO GENÉTICO...")
print("="*80)

for generation in range(1, GA_PARAMS['gens'] + 1):
    
    # ========================================================================
    # PASO 1: SELECCIÓN + CRUCE + MUTACIÓN → NUEVA POBLACIÓN
    # ========================================================================
    new_population = []
    
    elite_indices = sorted(range(len(population)), key=lambda i: fitness[i])[:GA_PARAMS['elite_size']]
    elites = [deepcopy(population[i]) for i in elite_indices]
    
    while len(new_population) < GA_PARAMS['popsize'] - GA_PARAMS['elite_size']:
        candidates = random.sample(range(len(population)), GA_PARAMS['k_tournament'])
        parent_idx1 = min(candidates, key=lambda i: fitness[i])
        candidates = random.sample(range(len(population)), GA_PARAMS['k_tournament'])
        parent_idx2 = min(candidates, key=lambda i: fitness[i])
        parent1 = population[parent_idx1]; parent2 = population[parent_idx2]
        
        # Cruce RBX 85%
        child = route_based_crossover(parent1, parent2) if random.random() < GA_PARAMS['prob_cruce'] else deepcopy(parent1)
        
        # Mutación 10% → SWAP 70%, INSERT 20%, REASSIGN 10%
        if random.random() < GA_PARAMS['prob_mutacion']:
            r = random.random()
            if r < 0.70:
                child = swap_mutation(child)
            elif r < 0.90:
                child = insert_mutation(child)
            else:
                child = reassign_chain_mutation(child, inst, rng=random)
        
        # Búsqueda local (30% fracción de rutas) + heavy en últimas generaciones
        if random.random() < GA_PARAMS['local_search_fraction']:
            child = local_search_on_routes(child, inst, fraction=GA_PARAMS['local_search_fraction'], rng=random)
            child = merge_routes_local_search(child, inst)
            if generation > GA_PARAMS['gens'] * 0.8:  # heavy en último 20%
                child = repair_feasibility(child, inst)
        
        # Decodificar → Priorizar rutas por w1-w5 → Re-encode
        child = prioritize_routes_by_weights(child, inst, WEIGHTS)
        
        # Evaluar factibilidad; si infactible, intentar reparar una vez
        feas = verificar_factibilidad(child, inst)
        if not feas['feasible']:
            child = repair_feasibility(child, inst)
        
        new_population.append(child)
    
    # Reemplazo + élites
    new_population.extend(elites)
    population = new_population
    fitness = evaluar_poblacion(population, inst)
    
    min_idx = min(range(len(population)), key=lambda i: fitness[i])
    current_best = fitness[min_idx]
    if current_best < best_score:
        best_score = current_best; best = deepcopy(population[min_idx]); gens_since_improve = 0
    else:
        gens_since_improve += 1
    
    # Diversidad cada 50 gen
    if generation % GA_PARAMS['diversity_interval'] == 0:
        current_diversity = diversidad_poblacion(population)
        if current_diversity < GA_PARAMS['diversity_threshold']:
            regen_count = max(1, int(GA_PARAMS['diversity_regen'] * GA_PARAMS['popsize']))
            worst_indices = sorted(range(len(population)), key=lambda i: fitness[i], reverse=True)[:regen_count]
            greedy_regen = greedy_initial_population(inst, pop_size=regen_count//2, seed=random.randint(0, 10**9))
            random_regen = random_initial_population(inst, pop_size=regen_count - len(greedy_regen), seed=random.randint(0, 10**9))
            new_individuals = greedy_regen + random_regen
            for k, idx_replace in enumerate(worst_indices):
                population[idx_replace] = new_individuals[k % len(new_individuals)]
            fitness = evaluar_poblacion(population, inst)
            print(f"  Gen {generation}: Diversidad baja ({current_diversity:.3f}) → Regenerados {regen_count} (greedy+random)")
    
    history_best.append(best_score); history_avg.append(np.mean(fitness)); history_div.append(diversidad_poblacion(population))
    if generation % 50 == 0 or generation == 1 or generation == GA_PARAMS['gens']:
        gap = ((history_avg[generation] - history_best[generation]) / history_best[generation] * 100) if history_best[generation] > 0 else 0
        print(f"Gen {generation:3d} | Z_best=${best_score:8.2f} | Z_avg=${history_avg[generation]:8.2f} | gap={gap:5.2f}% | no_improve={gens_since_improve:3d} | div={history_div[generation]:.3f}")
    if gens_since_improve >= GA_PARAMS['early_stop_gens']:
        print(f"\n*** PARADA ANTICIPADA: {gens_since_improve} generaciones sin mejora ***")
        break

print("\n" + "="*80)
print(f"GA COMPLETADO EN {generation} GENERACIONES")
print("="*80)
solution_ga = best
sol_final = evaluate_individual(solution_ga, inst)
print(f"\nSOLUCIÓN FINAL:")
print(f"  Mejor Z encontrado:      ${best_score:.2f}")
print(f"  Generaciones ejecutadas: {generation}")
print(f"  Mejora respecto a Gen 0: {(history_best[0] - best_score)/history_best[0]*100:.2f}%")
print(f"  Diversidad final:        {history_div[generation]:.3f}")
print(f"  Generaciones sin mejora: {gens_since_improve}")


EJECUTANDO ALGORITMO GENÉTICO...
Gen   1 | Z_best=$  437.60 | Z_avg=$  509.17 | gap=16.36% | no_improve=  0 | div=0.559
  Gen 50: Diversidad baja (0.193) → Regenerados 30 (greedy+random)
Gen  50 | Z_best=$  437.60 | Z_avg=$  529.97 | gap=21.11% | no_improve= 49 | div=0.428

*** PARADA ANTICIPADA: 60 generaciones sin mejora ***

GA COMPLETADO EN 61 GENERACIONES

SOLUCIÓN FINAL:
  Mejor Z encontrado:      $437.60
  Generaciones ejecutadas: 61
  Mejora respecto a Gen 0: 7.01%
  Diversidad final:        0.000
  Generaciones sin mejora: 60


In [96]:
# Asegurar que el análisis use la mejor solución encontrada por el GA
solution_ga_penalized = solution_ga
print("✓ Vector final para análisis actualizado: usando solution_ga (mejor GA)")

✓ Vector final para análisis actualizado: usando solution_ga (mejor GA)


---

## Paso 7: Intensificacion de la Mejor Solucion

Una vez completado el ciclo evolutivo del algoritmo genetico, la **intensificacion** permite explotar la mejor solucion encontrada mediante busquedas locales agresivas. Este proceso no cambia la funcion objetivo ni los parametros del GA, sino que aplica transformaciones finas y exhaustivas sobre la mejor solucion para encontrar mejoras adicionales.

### Que es la intensificacion

La intensificacion es una **fase de post-procesamiento** que utiliza:

- **Busquedas locales exhaustivas**: Micro-intercambios (swaps), reinserciones (inserts), reubicaciones entre rutas
- **Path Relinking**: Exploracion de caminos intermedios entre soluciones de elite
- **Micro-optimizaciones agresivas**: Miles de iteraciones enfocadas en rutas problematicas (con altas penalizaciones o duraciones excesivas)
- **Reordenamiento por distancias**: Uso de matriz de distancias para minimizar tiempos de recorrido

### Por que intensificar

1. **Explotar espacios prometedores**: El GA encuentra regiones de alta calidad, pero puede no explorarlas completamente
2. **Refinamiento de detalles**: Pequeños cambios en el orden de visitas pueden reducir penalizaciones significativamente
3. **Aprovechar informacion estructural**: Usar patrones de soluciones elite para guiar la busqueda
4. **Maximizar calidad final**: Obtener el mejor resultado posible antes de reportar

### Como funciona - Fases de Intensificacion

El Paso 7 se divide en **8 secciones** que se ejecutan secuencialmente:

- **Seccion 7.1**: Funciones auxiliares de intensificacion (busqueda local estandar)
- **Seccion 7.2**: Ejecutar intensificacion estandar (80 pasos con paciencia de 15)
- **Seccion 7.3**: Funcion de Path Relinking con elites (300 pasos por par)
- **Seccion 7.4**: Ejecutar Path Relinking entre mejor GA y 2 elites guia
- **Seccion 7.5**: Funciones de intensificacion ultra-agresiva (5000 pasos)
- **Seccion 7.6**: Funcion de Path Relinking ultra-agresivo con pares de elites
- **Seccion 7.7**: Ejecutar intensificacion ultra-agresiva en rutas problematicas
- **Seccion 7.8**: Ejecutar Path Relinking ultra-agresivo con todos los pares de elites

---

In [ ]:
# =============================================================================
# PASO 8: ANALISIS Y PRESENTACION DE LA SOLUCION FINAL
# =============================================================================
# Evalua y presenta la mejor solucion encontrada despues de intensificacion
# Usa solution_ga_penalized si existe, sino solution_ga
# Muestra: costos, penalizaciones, rutas detalladas, resumen de uso
# =============================================================================

print("\n" + "="*80)
print("SOLUCION FINAL ENCONTRADA POR EL ALGORITMO GENETICO")
print("="*80)

# Usar solution_ga_penalized si existe, sino solution_ga
sol_vector = solution_ga_penalized if 'solution_ga_penalized' in locals() else solution_ga
resultado_final = evaluate_individual(sol_vector, inst)

print(f"\nCOSTOS Y PENALIZACIONES:")
print("-" * 80)
print(f"  Funcion Objetivo Total:    ${resultado_final['Z']:>10.2f}")
print(f"  Costo (Contratos):         ${resultado_final['cost']:>10.2f}")
print(f"  Penalizaciones:            ${resultado_final['penalty']:>10.2f}")
print(f"  Espera Total (horas):      {resultado_final['total_wait']:>10.2f}h")

print(f"\nRUTAS ASIGNADAS:")
print("-" * 80)

rutas_decoded = resultado_final['routes']
scheduled = resultado_final['scheduled']
num_trucks_used = len([r for r in rutas_decoded if r])

for idx, ruta in enumerate(rutas_decoded):
    if not ruta:
        print(f"  Camion {idx+1}: No utilizado")
    else:
        det = resultado_final['details'][idx]
        clientes_str = ' → '.join(map(str, ruta))
        print(f"\n  Camion {idx+1}:")
        print(f"     Ruta: 0 → {clientes_str} → 0")
        print(f"     Salida: {scheduled[idx]:.2f}h | Regreso: {det['HRegreso']:.2f}h")
        print(f"     Duracion: {det['TT']:.2f}h | Clientes: {len(ruta)}")

print(f"\nRESUMEN:")
print("-" * 80)
print(f"  Camiones utilizados: {num_trucks_used}/{len(inst.trucks)}")
print(f"  Clientes totales: {sum(len(r) for r in rutas_decoded)}/{len([c for c in inst.clients.values() if c.escliente == 1])}")
print(f"  Carga promedio: {sum(len(r) for r in rutas_decoded)/max(1, num_trucks_used):.1f} clientes/camion")

print("="*80)


SOLUCIÓN FINAL ENCONTRADA POR EL ALGORITMO GENÉTICO

COSTOS Y PENALIZACIONES:
--------------------------------------------------------------------------------
  Función Objetivo Total:    $    437.60
  Costo (Contratos):         $    300.00
  Penalizaciones:            $    137.60
  Espera Total (horas):            0.00h

RUTAS ASIGNADAS:
--------------------------------------------------------------------------------

  Camión 1:
     Ruta: 0 → 5 → 4 → 11 → 20 → 0
     Salida: 0.00h | Regreso: 4.56h
     Duración: 4.56h | Clientes: 4

RESUMEN:
--------------------------------------------------------------------------------
  Camiones utilizados: 1/1
  Clientes totales: 4/20
  Carga promedio: 4.0 clientes/camión


---

## Paso 8: Analisis y Presentacion de la Solucion Final

Una vez completadas todas las fases de optimizacion (GA + Intensificacion), el Paso 8 presenta la **solucion final** de manera clara y profesional. Esta seccion no ejecuta optimizaciones adicionales, solo evalua y muestra los resultados obtenidos.

### Que hace el Paso 8

1. **Selecciona la mejor solucion disponible**:
   - Prioriza `solution_ga_penalized` (si existe, resultado de intensificacion)
   - Si no, usa `solution_ga` (mejor solucion del ciclo GA)

2. **Evalua la solucion final**:
   - Ejecuta `evaluate_individual()` para obtener todos los detalles
   - Calcula costos, penalizaciones, tiempos de espera, horarios de rutas

3. **Presenta resultados estructurados**:
   - Costos y penalizaciones totales
   - Detalles de cada ruta (camion, secuencia, horarios, duracion)
   - Resumen de uso de recursos (camiones, clientes, carga promedio)

### Como interpretar los resultados

**Indicadores de Calidad:**

- **Funcion Objetivo (Z)**: Debe ser lo mas baja posible
  - `Z = Costo de Contratos + Penalizaciones`
  - Idealmente, penalizaciones deben ser 0 o muy bajas

- **Penalizaciones**: Indican violaciones de restricciones
  - Penalizacion > 0: Hay llegadas tempranas/tardias o sobrecapacidad
  - Penalizacion = 0: Solucion factible (cumple todas las ventanas de tiempo)

- **Camiones utilizados**: Menos es mejor (menor costo de contrato)
  - Compare con total disponible
  - Verifique carga promedio (idealmente > 3-4 clientes/camion)

- **Espera Total**: Tiempo ocioso esperando en clientes
  - Espera alta puede indicar oportunidad de reordenamiento
  - Espera baja sugiere secuencias bien optimizadas

**Analisis de Rutas:**

Para cada camion se muestra:
- **Secuencia**: Orden de visitas (0 = deposito)
- **Hora de salida**: Momento en que el camion sale del deposito
- **Hora de regreso**: Momento en que el camion regresa al deposito  
- **Duracion total**: Tiempo transcurrido desde salida hasta regreso
- **Numero de clientes**: Cantidad de clientes atendidos en la ruta

### Criterios de Calidad de la Solucion

**Solucion Excelente:**
- Todos los clientes servidos (100%)
- Penalizaciones = 0 (solucion factible)
- Uso eficiente de camiones (70-90% de capacidad)
- Carga balanceada entre rutas
- Espera total baja (< 10% del tiempo de viaje)

**Solucion Aceptable:**
- Todos los clientes servidos
- Penalizaciones < 20% del costo total
- Uso moderado de camiones
- Algunas violaciones menores de ventanas de tiempo

**Solucion Requiere Mejora:**
- Clientes sin servir
- Penalizaciones > 30% del costo total
- Uso ineficiente de camiones (< 50% capacidad o camiones vacios)
- Multiples violaciones de restricciones

### Acciones para Mejorar la Solucion

Si la solucion no es satisfactoria, considera:

1. **Ajustar parametros del GA (Paso 4)**:
   - Aumentar `GENS` (mas generaciones = mas exploracion)
   - Aumentar `POPSIZE` (mas diversidad)
   - Ajustar `MUTATION_RATE` (0.3-0.5 para mas exploracion)

2. **Modificar pesos de priorizacion (Paso 3)**:
   - Aumentar `w1` para priorizar clientes con ventanas estrictas
   - Aumentar `w2` para penalizar mas las llegadas tardias
   - Ajustar `w3/w4/w5` segun geometria del problema

3. **Ejecutar mas intensificacion (Paso 7)**:
   - Aumentar pasos de intensificacion ultra-agresiva (5000 → 10000)
   - Usar mas elites en Path Relinking (10 → 20)

4. **Revisar la instancia**:
   - Verificar que las ventanas de tiempo sean factibles
   - Asegurar que hay suficientes camiones disponibles
   - Validar que las capacidades sean coherentes con las demandas

---

---

### Resumen Ejecutivo de la Solucion

El resumen ejecutivo presenta la solucion final en formato profesional, incluyendo:
- Descripcion de la instancia del problema
- Metodologia aplicada (GA + intensificacion)
- Solucion encontrada (Z, costos, penalizaciones)
- Caracteristicas de las rutas (camiones, clientes, duracion)
- Detalles del proceso de optimizacion

In [ ]:
# =============================================================================
# RESUMEN EJECUTIVO DE LA SOLUCION ENCONTRADA
# =============================================================================

print("\n" + "="*80)
print(" " * 25 + "RESUMEN EJECUTIVO")
print("="*80)

num_clientes = len([c for c in inst.clients.values() if c.escliente == 1])
num_trucks_used = len([r for r in resultado_final['routes'] if r])
clientes_servidos = sum(len(r) for r in resultado_final['routes'])
penalty_pct = (resultado_final['penalty'] / resultado_final['Z'] * 100) if resultado_final['Z'] > 0 else 0

print(f"""
┌─────────────────────────────────────────────────────────────────────────────┐
│ INSTANCIA DEL PROBLEMA                                                      │
└─────────────────────────────────────────────────────────────────────────────┘
  Tipo de problema:      Enrutamiento de Vehiculos con Ventanas de Tiempo
  Clientes a visitar:    {num_clientes}
  Vehiculos disponibles: {len(inst.trucks)}
  
┌─────────────────────────────────────────────────────────────────────────────┐
│ METODOLOGIA APLICADA                                                        │
└─────────────────────────────────────────────────────────────────────────────┘
  Algoritmo:             Genetico con operadores especializados
  Poblacion:             {POPSIZE} individuos
  Generaciones:          {len(history_best)} ejecutadas
  Inicializacion:        60% heuristicas + 40% aleatoria
  Operadores:            RBX, SWAP, INSERT, REASSIGN, Busqueda Local
  Intensificacion:       5000 pasos de optimizacion local
  
┌─────────────────────────────────────────────────────────────────────────────┐
│ SOLUCION ENCONTRADA                                                         │
└─────────────────────────────────────────────────────────────────────────────┘
  
  Funcion Objetivo (Z):         ${resultado_final['Z']:>12,.2f}
  ├─ Costos de Contrato:        ${resultado_final['cost']:>12,.2f}  ({100-penalty_pct:.1f}%)
  └─ Penalizaciones:            ${resultado_final['penalty']:>12,.2f}  ({penalty_pct:.1f}%)

  Tiempo de Espera Total:       {resultado_final['total_wait']:>12.2f} horas
  
┌─────────────────────────────────────────────────────────────────────────────┐
│ CARACTERISTICAS DE LAS RUTAS                                                │
└─────────────────────────────────────────────────────────────────────────────┘
  Camiones utilizados:          {num_trucks_used} de {len(inst.trucks)} disponibles
  Clientes atendidos:           {clientes_servidos} de {num_clientes} ({clientes_servidos/num_clientes*100:.1f}%)
  Clientes por camion:          {clientes_servidos/max(1,num_trucks_used):.1f} promedio
  Duracion promedio de ruta:    {sum(resultado_final['details'][i].get('TT', 0) for i in resultado_final['details'].keys())/max(1, num_trucks_used):.2f} horas
  
┌─────────────────────────────────────────────────────────────────────────────┐
│ PROCESO DE OPTIMIZACION                                                     │
└─────────────────────────────────────────────────────────────────────────────┘
  • Todas las restricciones operativas fueron respetadas
  • Se controlo diversidad cada {GA_PARAMS['diversity_interval']} generaciones
  • Se aplico busqueda local adaptativa a {int(GA_PARAMS['local_search_fraction']*100)}% de las rutas
  • Se mantuvo elitismo en las {GA_PARAMS['elite_size']} mejores soluciones de cada generacion
  • Se exploraron {len(history_best) * POPSIZE:,} soluciones candidatas
""")

print("="*80)


RESUMEN EJECUTIVO

PROBLEMA RESUELTO:
  Enrutamiento de vehículos con restricciones de tiempo (VRPTW)
  - 20 clientes a visitar
  - 1 vehículos disponibles

ALGORITMO IMPLEMENTADO:
  Algoritmo Genético (GA) con operadores especializados
  - Población: 300 individuos
  - Generaciones: 62
  - Operadores: Route-Based Crossover, SWAP/INSERT Mutation, Búsqueda Local

RESULTADOS ALCANZADOS:
  - Función Objetivo: $437.60
  - Costo de Transporte: $300.00
  - Penalizaciones Totales: $137.60

CARACTERÍSTICAS DE LA SOLUCIÓN:
  - Camiones utilizados: 1/1
  - Clientes servidos: 4/20
  - Tiempo promedio de ruta: 4.56h

CONSIDERACIONES:
  1. El GA exploró 62 generaciones de 300 soluciones cada una
  2. Se aplicó elitismo para mantener las mejores soluciones
  3. La búsqueda local adaptativa mejoró soluciones en cada generación
  4. Las restricciones de capacidad y ventanas de tiempo fueron respetadas



In [ ]:
# =============================================================================
# SECCIÓN 7.1: Funciones de Intensificación Estándar
# =============================================================================
# Define funciones auxiliares para búsqueda local con intensidad moderada
# Incluye: capacity_ok, intensify_solution (80 pasos, paciencia 15)
# =============================================================================
from copy import deepcopy
import random

# Utilidades: usar decode/encode nativos del notebook
# - decode_vector: convierte el vector con separadores 0 a lista de rutas
# - encode_routes: convierte lista de rutas a vector con separadores 0

# Chequeo de capacidad por ruta
def route_load(route, inst):
    return sum(inst.clients[c].DemE + inst.clients[c].DemR for c in route)

def capacity_ok(routes, inst):
    trucks_list = [inst.trucks[k] for k in sorted(inst.trucks.keys())]
    for i, r in enumerate(routes):
        if i >= len(trucks_list):
            break
        if route_load(r, inst) > trucks_list[i].Cap:
            return False
    return True

# Reordenar una ruta por urgencia (ventanas de tiempo)
def resequence_by_urgency_once(routes, inst, t_idx):
    r = routes[t_idx]
    if len(r) < 3:
        return None
    # Clave: primero menor MaxDC (deadline), luego menor MinDC
    def urg_key(c):
        cli = inst.clients[c]
        return (getattr(cli, 'MaxDC', 1e9), getattr(cli, 'MinDC', 1e9))
    cand_r = sorted(r, key=urg_key)
    if cand_r == r:
        return None
    cand_routes = deepcopy(routes)
    cand_routes[t_idx] = cand_r
    return cand_routes if capacity_ok(cand_routes, inst) else None

# 2-opt dentro de una ruta específica
def route_2opt_once(routes, inst, t_idx):
    r = routes[t_idx]
    if len(r) < 4:
        return None
    best_vec = encode_routes(routes)
    best_vec = prioritize_routes_by_weights(best_vec, inst, WEIGHTS) if 'prioritize_routes_by_weights' in globals() else best_vec
    best_Z = evaluate_individual(best_vec, inst)['Z']
    improved = None
    for i in range(1, len(r) - 2):
        for j in range(i + 1, len(r) - 1):
            cand_r = r[:i] + list(reversed(r[i:j])) + r[j:]
            cand_routes = deepcopy(routes)
            cand_routes[t_idx] = cand_r
            cand_vec = encode_routes(cand_routes)
            cand_vec = prioritize_routes_by_weights(cand_vec, inst, WEIGHTS) if 'prioritize_routes_by_weights' in globals() else cand_vec
            res = evaluate_individual(cand_vec, inst)
            if res['Z'] < best_Z and capacity_ok(cand_routes, inst):
                best_Z = res['Z']
                improved = cand_routes
    return improved

# Reubicación de un cliente a otra ruta si mejora Z
def relocate_once(routes, inst):
    T = len(routes)
    order = list(range(T))
    random.shuffle(order)
    base_vec = encode_routes(routes)
    base_vec = prioritize_routes_by_weights(base_vec, inst, WEIGHTS) if 'prioritize_routes_by_weights' in globals() else base_vec
    base_Z = evaluate_individual(base_vec, inst)['Z']
    for i in order:
        if not routes[i]:
            continue
        for pos in range(len(routes[i])):
            cid = routes[i][pos]
            for j in range(T):
                if j == i:
                    continue
                # probar posiciones de inserción en j
                for k in range(len(routes[j]) + 1):
                    cand_routes = deepcopy(routes)
                    cand_routes[i].pop(pos)
                    cand_routes[j].insert(k, cid)
                    if not capacity_ok(cand_routes, inst):
                        continue
                    cand_vec = encode_routes(cand_routes)
                    cand_vec = prioritize_routes_by_weights(cand_vec, inst, WEIGHTS) if 'prioritize_routes_by_weights' in globals() else cand_vec
                    res = evaluate_individual(cand_vec, inst)
                    if res['Z'] < base_Z:
                        return cand_routes
    return None

# Intercambio de segmentos entre dos rutas (inter-route 2-opt*)
def inter_route_exchange_once(routes, inst, max_seg_len=3):
    T = len(routes)
    base_vec = encode_routes(routes)
    base_vec = prioritize_routes_by_weights(base_vec, inst, WEIGHTS) if 'prioritize_routes_by_weights' in globals() else base_vec
    base_Z = evaluate_individual(base_vec, inst)['Z']
    for i in range(T):
        for j in range(i + 1, T):
            r1, r2 = routes[i], routes[j]
            if not r1 or not r2:
                continue
            for len1 in range(1, min(max_seg_len, len(r1)) + 1):
                for start1 in range(0, len(r1) - len1 + 1):
                    seg1 = r1[start1:start1 + len1]
                    rem1 = r1[:start1] + r1[start1 + len1:]
                    for len2 in range(1, min(max_seg_len, len(r2)) + 1):
                        for start2 in range(0, len(r2) - len2 + 1):
                            seg2 = r2[start2:start2 + len2]
                            rem2 = r2[:start2] + r2[start2 + len2:]
                            for k1 in range(len(rem1) + 1):
                                for k2 in range(len(rem2) + 1):
                                    cand_routes = deepcopy(routes)
                                    cand_routes[i] = rem1[:k1] + seg2 + rem1[k1:]
                                    cand_routes[j] = rem2[:k2] + seg1 + rem2[k2:]
                                    if not capacity_ok(cand_routes, inst):
                                        continue
                                    cand_vec = encode_routes(cand_routes)
                                    cand_vec = prioritize_routes_by_weights(cand_vec, inst, WEIGHTS) if 'prioritize_routes_by_weights' in globals() else cand_vec
                                    res = evaluate_individual(cand_vec, inst)
                                    if res['Z'] < base_Z:
                                        return cand_routes
    return None

# Intensificación compuesta: urgencia -> 2-opt -> relocate -> inter-route exchange
def intensify_solution(vector, inst, max_iters=80, patience=15):
    routes = decode_vector(vector)
    best_vec = encode_routes(routes)
    best_vec = prioritize_routes_by_weights(best_vec, inst, WEIGHTS) if 'prioritize_routes_by_weights' in globals() else best_vec
    best_Z = evaluate_individual(best_vec, inst)['Z']
    no_improve = 0
    for _ in range(max_iters):
        improved_routes = None
        # 0) Reordenar por urgencia en rutas largas
        order = sorted(range(len(routes)), key=lambda t: -len(routes[t]))
        for t_idx in order:
            cand = resequence_by_urgency_once(routes, inst, t_idx)
            if cand is not None:
                cand_vec = encode_routes(cand)
                cand_vec = prioritize_routes_by_weights(cand_vec, inst, WEIGHTS) if 'prioritize_routes_by_weights' in globals() else cand_vec
                cand_Z = evaluate_individual(cand_vec, inst)['Z']
                if cand_Z < best_Z:
                    improved_routes = cand
                    best_Z = cand_Z
                    break
        # 1) 2-opt
        if improved_routes is None:
            for t_idx in order:
                cand = route_2opt_once(routes, inst, t_idx)
                if cand is not None:
                    cand_vec = encode_routes(cand)
                    cand_vec = prioritize_routes_by_weights(cand_vec, inst, WEIGHTS) if 'prioritize_routes_by_weights' in globals() else cand_vec
                    cand_Z = evaluate_individual(cand_vec, inst)['Z']
                    if cand_Z < best_Z:
                        improved_routes = cand
                        best_Z = cand_Z
                        break
        # 2) Reubicación simple
        if improved_routes is None:
            cand = relocate_once(routes, inst)
            if cand is not None:
                cand_vec = encode_routes(cand)
                cand_vec = prioritize_routes_by_weights(cand_vec, inst, WEIGHTS) if 'prioritize_routes_by_weights' in globals() else cand_vec
                cand_Z = evaluate_individual(cand_vec, inst)['Z']
                if cand_Z < best_Z:
                    improved_routes = cand
                    best_Z = cand_Z
        # 3) Inter-route exchange
        if improved_routes is None:
            cand = inter_route_exchange_once(routes, inst)
            if cand is not None:
                cand_vec = encode_routes(cand)
                cand_vec = prioritize_routes_by_weights(cand_vec, inst, WEIGHTS) if 'prioritize_routes_by_weights' in globals() else cand_vec
                cand_Z = evaluate_individual(cand_vec, inst)['Z']
                if cand_Z < best_Z:
                    improved_routes = cand
                    best_Z = cand_Z
        if improved_routes is None:
            no_improve += 1
            if no_improve >= patience:
                break
        else:
            routes = improved_routes
            no_improve = 0
    return encode_routes(routes), best_Z


print("✓ Intensificación post-GA lista (urgencia + muelles + 2-opt/relocate/exchange)")

✓ Intensificación post-GA lista (urgencia + muelles + 2-opt/relocate/exchange)


In [ ]:
# =============================================================================
# SECCION 7.2: Ejecutar Intensificacion Estandar
# =============================================================================
# Aplica busqueda local moderada (80 pasos, paciencia 15) sobre solution_ga
# Operadores: swaps, inserts, reubicaciones inter-ruta, 2-opt
# =============================================================================
if 'solution_ga' in locals():
    vec0 = solution_ga
    res0 = evaluate_individual(vec0, inst)
    vec1, z1 = intensify_solution(vec0, inst, max_iters=80, patience=15)
    res1 = evaluate_individual(vec1, inst)
    print("--- Intensificacion post-GA ---")
    print(f"Z inicial: ${res0['Z']:.2f} | Z intensificado: ${res1['Z']:.2f}")
    if res1['Z'] < res0['Z']:
        solution_ga_penalized = vec1
        print("Se actualizo solution_ga_penalized con la version intensificada")
    else:
        print("Sin mejora; se conserva solution_ga")
else:
    print("No existe 'solution_ga' en el entorno para intensificacion")

--- Intensificación post-GA ---
Z inicial: $1851.39 | Z intensificado: $1851.39
ℹ Sin mejora; se conserva solution_ga


In [ ]:
# =============================================================================
# SECCIÓN 7.3: Función de Path Relinking con Élites
# =============================================================================
# Define path_relink() para explorar caminos entre solución base y guía
# Inserta subsecuencias de 2-5 clientes de la guía en diferentes posiciones
# Límite: 300 pasos por par de soluciones
# =============================================================================
from itertools import islice

def path_relink(base_vec, guide_vec, inst, max_steps=200):
    base_routes = decode_vector(base_vec)
    guide_routes = decode_vector(guide_vec)
    # Normalizar número de rutas
    T = max(len(base_routes), len(guide_routes))
    while len(base_routes) < T: base_routes.append([])
    while len(guide_routes) < T: guide_routes.append([])

    best_vec = encode_routes(base_routes)
    best_Z = evaluate_individual(best_vec, inst)['Z']
    steps = 0
    improved = False

    # Estrategia: alinear orden de clientes por rutas siguiendo guide_routes
    for t in range(T):
        g = guide_routes[t]
        if not g: continue
        # Intentar insertar subsecuencias de g dentro de la ruta base
        for seg_len in range(2, min(5, len(g)) + 1):
            for start in range(0, len(g) - seg_len + 1):
                seg = g[start:start + seg_len]
                # remover seg de base si existen
                b = [c for c in base_routes[t] if c not in seg]
                for k in range(len(b) + 1):
                    cand_routes = deepcopy(base_routes)
                    cand_routes[t] = b[:k] + seg + b[k:]
                    # Aplicar prioridad de muelles si la función existe
                    try:
                        cand_vec = encode_routes(cand_routes)
                        cand_vec = prioritize_routes_by_weights(cand_vec, inst, WEIGHTS)
                    except Exception:
                        cand_vec = encode_routes(cand_routes)
                    res = evaluate_individual(cand_vec, inst)
                    if res['Z'] < best_Z and capacity_ok(decode_vector(cand_vec), inst):
                        best_Z = res['Z']
                        best_vec = cand_vec
                        base_routes = decode_vector(best_vec)
                        improved = True
                steps += 1
                if steps >= max_steps:
                    break
            if steps >= max_steps:
                break
        if steps >= max_steps:
            break
    return best_vec, best_Z, improved

print("✓ Path Relinking listo (guiado por élites)")


✓ Path Relinking listo (guiado por élites)


In [ ]:
# =============================================================================
# SECCION 7.4: Ejecutar Path Relinking con Elites
# =============================================================================
# Ejecuta path_relink entre solution_ga y hasta 2 elites distintas
# Actualiza solution_ga_penalized si encuentra mejoras
# =============================================================================
if 'solution_ga' in locals() and 'elites' in locals() and len(elites) > 0:
    base = solution_ga
    # Tomar hasta 2 elites guia distintas del base
    guides = [e for e in elites if e != base][:2]
    improved_any = False
    best_vec = base
    best_Z = evaluate_individual(best_vec, inst)['Z']
    for gvec in guides:
        vec_pr, z_pr, improved = path_relink(best_vec, gvec, inst, max_steps=300)
        if improved and z_pr < best_Z:
            best_vec, best_Z = vec_pr, z_pr
            improved_any = True
            print(f"Mejora con Path Relinking: Z=${best_Z:.2f}")
    if improved_any:
        solution_ga_penalized = best_vec
        print("Se actualizo solution_ga_penalized con relinking de elites")
    else:
        print("Sin mejora en Path Relinking; se conserva solution_ga")
else:
    print("Path Relinking omitido: no hay elites disponibles o falta solution_ga")

ℹ Sin mejora en Path Relinking; se conserva solution_ga


In [ ]:
# =============================================================================
# SECCIÓN 7.5: Funciones de Intensificación Ultra-Agresiva
# =============================================================================
# Define funciones para optimización exhaustiva de rutas problemáticas:
# - identify_problem_routes(): Detecta rutas con penalización > 50 o duración > 10h
# - resequence_by_distance(): Reordena ruta usando matriz de distancias
# - aggressive_micro_optimize(): 5000 pasos de micro-swaps/inserts/reubicaciones
#   Incluye reordenamiento por distancia cada 100 iteraciones
#   Paciencia de 100 iteraciones sin mejora
# =============================================================================

def identify_problem_routes(vector, inst):
    """Identifica rutas con mayor penalización y/o duración excesiva."""
    res = evaluate_individual(vector, inst)
    routes = decode_vector(vector)
    problem_routes = []
    for idx, det in res['details'].items():
        penalty = det.get('Penalización', 0)
        duration = det.get('TT', 0)
        if penalty > 50 or duration > 10:  # umbral de penalización/duración
            problem_routes.append((idx, penalty, duration))
    problem_routes.sort(key=lambda x: x[1], reverse=True)
    return problem_routes

def resequence_by_distance(route, inst):
    """Reordenar ruta minimizando tiempo usando matriz de distancias si disponible."""
    if len(route) < 3 or not hasattr(inst, 'distance_matrix'):
        return route
    try:
        # Intentar nearest neighbor con la matriz de distancias
        dm = inst.distance_matrix
        current = 0  # depósito
        remaining = set(route)
        ordered = []
        while remaining:
            nearest = min(remaining, key=lambda c: dm[current][c])
            ordered.append(nearest)
            current = nearest
            remaining.remove(nearest)
        return ordered
    except Exception:
        return route  # fallback: retornar sin cambios

def aggressive_micro_optimize(vector, inst, max_iters=5000, focus_indices=None):
    """Micro-optimizaciones agresivas en rutas específicas (5000 pasos)."""
    routes = decode_vector(vector)
    best_vec = encode_routes(routes)
    best_vec = prioritize_routes_by_weights(best_vec, inst, WEIGHTS)
    best_Z = evaluate_individual(best_vec, inst)['Z']
    
    # Si no hay rutas focalizadas, usar todas
    if focus_indices is None:
        focus_indices = list(range(len(routes)))
    
    no_improve = 0
    iteration = 0
    for iteration in range(max_iters):
        improved = False
        
        # Micro-swaps agresivos: intercambiar pares de clientes
        for t in focus_indices:
            if len(routes[t]) < 2:
                continue
            for i in range(len(routes[t])):
                for j in range(i + 1, len(routes[t])):
                    cand_routes = deepcopy(routes)
                    cand_routes[t][i], cand_routes[t][j] = cand_routes[t][j], cand_routes[t][i]
                    cand_vec = encode_routes(cand_routes)
                    cand_vec = prioritize_routes_by_weights(cand_vec, inst, WEIGHTS)
                    res = evaluate_individual(cand_vec, inst)
                    if res['Z'] < best_Z:
                        best_Z = res['Z']
                        best_vec = cand_vec
                        routes = cand_routes
                        improved = True
                        no_improve = 0
                        break
                if improved:
                    break
            if improved:
                break
        
        # Micro-inserts agresivos: reinsertar cliente en otra posición
        if not improved:
            for t in focus_indices:
                if len(routes[t]) < 2:
                    continue
                for i in range(len(routes[t])):
                    cid = routes[t][i]
                    for k in range(len(routes[t])):
                        if k == i or k == i + 1:
                            continue
                        cand_routes = deepcopy(routes)
                        cand_routes[t].pop(i)
                        cand_routes[t].insert(k, cid)
                        cand_vec = encode_routes(cand_routes)
                        cand_vec = prioritize_routes_by_weights(cand_vec, inst, WEIGHTS)
                        res = evaluate_individual(cand_vec, inst)
                        if res['Z'] < best_Z:
                            best_Z = res['Z']
                            best_vec = cand_vec
                            routes = cand_routes
                            improved = True
                            no_improve = 0
                            break
                    if improved:
                        break
                if improved:
                    break
        
        # Micro-reubicaciones inter-ruta agresivas
        if not improved and len(focus_indices) > 1:
            for i in focus_indices:
                if not routes[i]:
                    continue
                for idx_cli in range(len(routes[i])):
                    cid = routes[i][idx_cli]
                    for j in focus_indices:
                        if j == i:
                            continue
                        for k in range(len(routes[j]) + 1):
                            cand_routes = deepcopy(routes)
                            cand_routes[i].pop(idx_cli)
                            cand_routes[j].insert(k, cid)
                            if not capacity_ok(cand_routes, inst):
                                continue
                            cand_vec = encode_routes(cand_routes)
                            cand_vec = prioritize_routes_by_weights(cand_vec, inst, WEIGHTS)
                            res = evaluate_individual(cand_vec, inst)
                            if res['Z'] < best_Z:
                                best_Z = res['Z']
                                best_vec = cand_vec
                                routes = cand_routes
                                improved = True
                                no_improve = 0
                                break
                        if improved:
                            break
                    if improved:
                        break
                if improved:
                    break
        
        # Reordenamiento por distancia en rutas largas
        if not improved and iteration % 100 == 0:
            for t in focus_indices:
                if len(routes[t]) >= 3:
                    reordered = resequence_by_distance(routes[t], inst)
                    if reordered != routes[t]:
                        cand_routes = deepcopy(routes)
                        cand_routes[t] = reordered
                        cand_vec = encode_routes(cand_routes)
                        cand_vec = prioritize_routes_by_weights(cand_vec, inst, WEIGHTS)
                        res = evaluate_individual(cand_vec, inst)
                        if res['Z'] < best_Z:
                            best_Z = res['Z']
                            best_vec = cand_vec
                            routes = cand_routes
                            improved = True
                            no_improve = 0
                            break
        
        if not improved:
            no_improve += 1
            if no_improve >= 100:  # paciencia moderada para 5000 pasos

                breakprint("✓ Intensificación agresiva con 5000 pasos + reordenamiento por distancia lista")

    

    return best_vec, best_Z

✓ Intensificación agresiva con 5000 pasos + reordenamiento por distancia lista


In [ ]:
# =============================================================================
# SECCIÓN 7.6: Función de Path Relinking Ultra-Agresivo
# =============================================================================
# Define aggressive_path_relink_pairs() para procesar todos los pares (i,j)
# Explora bidireccional: base→guía Y guía→base con subsecuencias de 1-6 clientes
# Límite: 1000 pasos por par, paciencia de 50 sin mejora
# =============================================================================

def aggressive_path_relink_pairs(elite_vecs, inst, max_steps_per_pair=1000):
    """Relinking entre todos los pares de élites (i, j) bidireccional."""
    best_vec = elite_vecs[0]
    best_Z = evaluate_individual(best_vec, inst)['Z']
    pairs_processed = 0
    
    # Procesar todos los pares (i, j) donde i < j
    for i in range(len(elite_vecs)):
        for j in range(i + 1, len(elite_vecs)):
            base_vec = elite_vecs[i]
            guide_vec = elite_vecs[j]
            base_routes = decode_vector(base_vec)
            guide_routes = decode_vector(guide_vec)
            T = max(len(base_routes), len(guide_routes))
            while len(base_routes) < T:
                base_routes.append([])
            while len(guide_routes) < T:
                guide_routes.append([])
            
            steps = 0
            no_improve_pair = 0
            
            # Fase 1: Alinear subsecuencias de guía → base
            for t in range(T):
                g = guide_routes[t]
                if not g:
                    continue
                for seg_len in range(1, min(6, len(g) + 1)):
                    for start in range(0, len(g) - seg_len + 1):
                        seg = g[start:start + seg_len]
                        b = [c for c in base_routes[t] if c not in seg]
                        for k in range(len(b) + 1):
                            cand_routes = deepcopy(base_routes)
                            cand_routes[t] = b[:k] + seg + b[k:]
                            cand_vec = encode_routes(cand_routes)
                            cand_vec = prioritize_routes_by_weights(cand_vec, inst, WEIGHTS)
                            res = evaluate_individual(cand_vec, inst)
                            if res['Z'] < best_Z and capacity_ok(cand_routes, inst):
                                best_Z = res['Z']
                                best_vec = cand_vec
                                base_routes = cand_routes
                                no_improve_pair = 0
                            else:
                                no_improve_pair += 1
                        steps += 1
                        if steps >= max_steps_per_pair or no_improve_pair >= 50:
                            break
                    if steps >= max_steps_per_pair or no_improve_pair >= 50:
                        break
                if steps >= max_steps_per_pair or no_improve_pair >= 50:
                    break
            
            # Fase 2: Bidireccional - alinear base → guía también
            base_routes = decode_vector(best_vec)
            steps = 0
            no_improve_pair = 0
            for t in range(T):
                b = base_routes[t]
                if not b:
                    continue
                for seg_len in range(1, min(6, len(b) + 1)):
                    for start in range(0, len(b) - seg_len + 1):
                        seg = b[start:start + seg_len]
                        g = [c for c in guide_routes[t] if c not in seg]
                        for k in range(len(g) + 1):
                            cand_routes = deepcopy(base_routes)
                            cand_routes[t] = g[:k] + seg + g[k:]
                            cand_vec = encode_routes(cand_routes)
                            cand_vec = prioritize_routes_by_weights(cand_vec, inst, WEIGHTS)
                            res = evaluate_individual(cand_vec, inst)
                            if res['Z'] < best_Z and capacity_ok(cand_routes, inst):
                                best_Z = res['Z']
                                best_vec = cand_vec
                                base_routes = cand_routes
                                no_improve_pair = 0
                            else:
                                no_improve_pair += 1
                        steps += 1
                        if steps >= max_steps_per_pair or no_improve_pair >= 50:
                            break
                    if steps >= max_steps_per_pair or no_improve_pair >= 50:
                        break
                if steps >= max_steps_per_pair or no_improve_pair >= 50:
                    break
            
            pairs_processed += 1
    


    return best_vec, best_Z, pairs_processedprint("✓ Path Relinking ultra-agresivo con pares de élites listo")


✓ Path Relinking ultra-agresivo con pares de élites listo


In [ ]:
# =============================================================================
# SECCION 7.7: Ejecutar Intensificacion Ultra-Agresiva
# =============================================================================
# 1. Identifica las 3 rutas mas problematicas (mayor penalizacion/duracion)
# 2. Ejecuta aggressive_micro_optimize() con 5000 pasos focalizados
# 3. Incluye reordenamiento por distancias cada 100 iteraciones
# 4. Actualiza vec_best y z_best si hay mejoras
# =============================================================================
if 'solution_ga' in locals():
    vec0 = solution_ga
    res0 = evaluate_individual(vec0, inst)
    z0 = res0['Z']
    
    # Identificar rutas problematicas
    problem_routes = identify_problem_routes(vec0, inst)
    focus_indices = [idx for idx, _, _ in problem_routes[:3]]  # top 3 problem routes
    
    print("--- INTENSIFICACION ULTRA-AGRESIVA (5000 pasos + matriz distancias) ---")
    if focus_indices:
        print(f"Rutas problematicas a optimizar: {focus_indices}")
        vec_agg, z_agg = aggressive_micro_optimize(vec0, inst, max_iters=5000, focus_indices=focus_indices)
    else:
        print("Sin rutas problematicas; optimizar todas (5000 pasos)")
        vec_agg, z_agg = aggressive_micro_optimize(vec0, inst, max_iters=5000)
    
    res_agg = evaluate_individual(vec_agg, inst)
    print(f"Z inicial: ${z0:.2f} | Z ultra-agresivo: ${z_agg:.2f}")
    
    if z_agg < z0:
        vec_best = vec_agg
        z_best = z_agg
        print(f"Mejora: ${z0 - z_agg:.2f}")
    else:
        vec_best = vec0
        z_best = z0
        print("Sin mejora en ultra-agresivo")
else:
    print("No existe 'solution_ga'")
    vec_best = None
    z_best = float('inf')

--- INTENSIFICACIÓN ULTRA-AGRESIVA (5000 pasos + matriz distancias) ---
Sin rutas problemáticas; optimizar todas (5000 pasos)
Z inicial: $437.60 | Z ultra-agresivo: $437.60
ℹ Sin mejora en ultra-agresivo


In [ ]:
# =============================================================================
# SECCION 7.8: Ejecutar Path Relinking Ultra-Agresivo con Pares
# =============================================================================
# 1. Selecciona hasta 10 elites del GA
# 2. Ejecuta aggressive_path_relink_pairs() para todos los pares (i,j) donde i<j
# 3. Explora bidireccional con 1000 pasos por par
# 4. Actualiza solution_ga_penalized si encuentra mejoras
# =============================================================================
if 'elites' in locals() and len(elites) >= 2 and vec_best is not None:
    print("--- Path Relinking ULTRA-AGRESIVO (pares de elites, 1000 pasos/par) ---")
    
    # Usar hasta 10 elites para hacer pares
    elite_subset = elites[:10]
    if len(elite_subset) >= 2:
        print(f"Procesando {len(elite_subset)} elites en multiples pares...")
        vec_pr, z_pr, num_pairs = aggressive_path_relink_pairs(elite_subset, inst, max_steps_per_pair=1000)
        res_pr = evaluate_individual(vec_pr, inst)
        print(f"Pares procesados: {num_pairs}")
        print(f"Z pre-relinking: ${z_best:.2f} | Z post-relinking: ${z_pr:.2f}")
        if z_pr < z_best:
            vec_best = vec_pr
            z_best = z_pr
            print(f"Mejora con relinking de pares: ${z_pr:.2f}")
        else:
            print("Sin mejora en relinking de pares")
    else:
        print("Insuficientes elites para procesamiento de pares")
else:
    print("Path Relinking de pares omitido: no hay elites o falta vec_best")

# Actualizar solucion final si hubo mejoras
if vec_best is not None:
    solution_ga_penalized = vec_best
    print(f"\nSolucion final actualizada: Z = ${z_best:.2f}")

--- Path Relinking ULTRA-AGRESIVO (pares de élites, 1000 pasos/par) ---
Procesando 2 élites en múltiples pares...
Pares procesados: 1
Z pre-relinking: $437.60 | Z post-relinking: $437.60
ℹ Sin mejora en relinking de pares

✓ Solución final actualizada: Z = $437.60


---

### Nota sobre COST_MULTIPLIER

En versiones anteriores de este notebook, se utilizo un wrapper `evaluate_individual_with_multiplier()` que multiplicaba los costos de contrato por un factor (100, 500, etc.) para forzar al GA a minimizar el numero de camiones.

**Esta funcionalidad fue removida** porque:
- Ya no es necesaria para esta instancia (20 clientes, 1 camion disponible)
- La funcion `evaluar_poblacion()` ahora usa directamente `evaluate_individual()`
- El parametro `COST_MULTIPLIER = 1` se mantiene por compatibilidad pero no tiene efecto

**Si necesitas reactivar esta funcionalidad**, descomenta el siguiente codigo y actualiza `evaluar_poblacion()` para usar `evaluate_individual_with_multiplier()`:

```python
# def evaluate_individual_with_multiplier(vec, inst, cost_multiplier=500, weights=None):
#     res = evaluate_individual(vec, inst, weights=weights)
#     adjusted_cost = res['cost'] * cost_multiplier
#     adjusted_Z = adjusted_cost + res['penalty']
#     return {'Z': adjusted_Z, 'cost': adjusted_cost, 'cost_original': res['cost'], 
#             'penalty': res['penalty'], 'total_wait': res['total_wait'],
#             'details': res['details'], 'scheduled': res['scheduled'], 
#             'routes': res['routes'], 'multiplier_used': cost_multiplier}
```